In [1]:
import numpy as np 
import pandas as pd 

from PyQt5 import QtCore, QtGui, QtWidgets
import sys

import matplotlib.pyplot as plt

from PyQt5.QtGui import QPixmap

from PyQt5.QtCore import QDate, QDateTime

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge
from scipy.optimize import minimize

import warnings
warnings.filterwarnings("ignore")

In [2]:
def get_trained_models(X, Y):
    pipe = Pipeline([
        ('scaler', MinMaxScaler(feature_range=(0, 1))),
        ('poly', PolynomialFeatures(degree=5)),
        ('lr', Ridge(alpha=1.0))
    ])

    target_transform = MinMaxScaler(feature_range=(0, 1))
    
    target_transform.fit(Y)
    pipe.fit(X, target_transform.transform(Y))
    return pipe, target_transform

def minimize_maximize(pipe, target_transform, boundary):
    startpoint = np.array([np.mean(b) for b in boundary])
    min_functions = []
    min_values = []
    for i in range(len(target_transform.data_range_)):
        min_functions.append(lambda x: pipe.predict([x])[:, i])
        min_values.append(target_transform.inverse_transform(pipe.predict([minimize(min_functions[i], x0=startpoint, bounds=boundary).x]))[0, i])
        
    max_functions = []
    max_values = []
    for i in range(len(target_transform.data_range_)):
        max_functions.append(lambda x: -pipe.predict([x])[:, i])    
        max_values.append(target_transform.inverse_transform(pipe.predict([minimize(max_functions[i], x0=startpoint, bounds=boundary).x]))[0, i])
    
    return [(min_v, max_v) for min_v, max_v in zip(min_values, max_values)]

def find_Xi_pareto(pipe, target_transform, X_boundary, Y_desired_boundary, target, steps=100):
    ls = [np.array([np.linspace(b[0], b[1], num=steps) if i != target else np.ones(steps)*v for i, b in enumerate(X_boundary)]).T for v in np.linspace(X_boundary[target][0], X_boundary[target][1], steps)]
    results = []
    for l in ls:
        preds = target_transform.inverse_transform(pipe.predict(l))
        results.append({'min': preds.min(axis=0), 'max': preds.max(axis=0)})

    filtered_results = []

    for r, value in zip(results, np.linspace(X_boundary[target][0], X_boundary[target][1], steps)):
        if all([r['min'][ind] >= y[0] and r['max'][ind] <= y[1] for ind, y in enumerate(Y_desired_boundary)]):
            filtered_results.append(value)
            
    return filtered_results

def find_X_pareto(pipe, target_transform, X_boundary, Y_desired_boundary, X_targets, steps=100, multipl=0.75, iteration=0):
    print(iteration)
    if iteration >= 10:
        return 'Empty'
    results = []
    for target in X_targets:
        results.append(find_Xi_pareto(pipe, target_transform, X_boundary, Y_desired_boundary, target, steps=100))
    if any([len(r) for r in results]):
        ind_corrected = np.argmax([len(r) for r in results])
        target_corrected = X_targets[ind_corrected]
        X_boundary_new = [b if ind != target_corrected else (results[ind_corrected][0], results[ind_corrected][-1]) for ind, b in enumerate(X_boundary)]
        return X_boundary_new
    else:
        X_boundary_new = [b if ind not in X_targets else (np.mean(b) - multipl*(np.mean(b) - b[0]), np.mean(b) + multipl*(b[1] - np.mean(b))) for ind, b in enumerate(X_boundary)]
        return find_X_pareto(pipe, target_transform, X_boundary_new, Y_desired_boundary, X_targets, steps=steps*multipl, multipl=multipl, iteration=iteration + 1)
    
def get_results(inp):
    X = inp['X_values']
    Y = inp['Y_values']
    X_boundary = [(x['min'], x['max']) for x in inp['X']]
    Y_desired_boundary = [(y['min'], y['max']) for y in inp['Y']]
    poly = np.quantile(inp['poly'], 0.75)
    action = inp['action']
    X_targets = np.where([x['correct'] for x in inp['X']])[0]
    
    pipe, target_transform = get_trained_models(X, Y)
    real_bounds = minimize_maximize(pipe, target_transform, X_boundary)
    if action == 'Y':
        return real_bounds
    else:
        return find_X_pareto(pipe, target_transform, X_boundary, Y_desired_boundary, X_targets, steps=100)

In [3]:
class Ui_MainWindow(object):
    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(671, 625)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        self.gridLayoutWidget_2 = QtWidgets.QWidget(self.centralwidget)
        self.gridLayoutWidget_2.setGeometry(QtCore.QRect(10, 470, 281, 91))
        self.gridLayoutWidget_2.setObjectName("gridLayoutWidget_2")
        self.gridLayout_2 = QtWidgets.QGridLayout(self.gridLayoutWidget_2)
        self.gridLayout_2.setContentsMargins(0, 0, 0, 0)
        self.gridLayout_2.setObjectName("gridLayout_2")
        self.lineEdit_13 = QtWidgets.QLineEdit(self.gridLayoutWidget_2)
        self.lineEdit_13.setObjectName("lineEdit_13")
        self.gridLayout_2.addWidget(self.lineEdit_13, 3, 1, 1, 1)
        self.lineEdit_16 = QtWidgets.QLineEdit(self.gridLayoutWidget_2)
        self.lineEdit_16.setObjectName("lineEdit_16")
        self.gridLayout_2.addWidget(self.lineEdit_16, 1, 2, 1, 1)
        self.lineEdit_14 = QtWidgets.QLineEdit(self.gridLayoutWidget_2)
        self.lineEdit_14.setObjectName("lineEdit_14")
        self.gridLayout_2.addWidget(self.lineEdit_14, 1, 1, 1, 1)
        self.checkBox_7 = QtWidgets.QCheckBox(self.gridLayoutWidget_2)
        self.checkBox_7.setChecked(True)
        self.checkBox_7.setObjectName("checkBox_7")
        self.gridLayout_2.addWidget(self.checkBox_7, 1, 0, 1, 1)
        self.lineEdit_15 = QtWidgets.QLineEdit(self.gridLayoutWidget_2)
        self.lineEdit_15.setObjectName("lineEdit_15")
        self.gridLayout_2.addWidget(self.lineEdit_15, 3, 2, 1, 1)
        self.checkBox_8 = QtWidgets.QCheckBox(self.gridLayoutWidget_2)
        self.checkBox_8.setChecked(True)
        self.checkBox_8.setObjectName("checkBox_8")
        self.gridLayout_2.addWidget(self.checkBox_8, 3, 0, 1, 1)
        self.label_6 = QtWidgets.QLabel(self.gridLayoutWidget_2)
        self.label_6.setObjectName("label_6")
        self.gridLayout_2.addWidget(self.label_6, 0, 1, 1, 1)
        self.gridLayoutWidget_3 = QtWidgets.QWidget(self.centralwidget)
        self.gridLayoutWidget_3.setGeometry(QtCore.QRect(140, 40, 111, 94))
        self.gridLayoutWidget_3.setObjectName("gridLayoutWidget_3")
        self.gridLayout_3 = QtWidgets.QGridLayout(self.gridLayoutWidget_3)
        self.gridLayout_3.setContentsMargins(0, 0, 0, 0)
        self.gridLayout_3.setObjectName("gridLayout_3")
        self.spinBox = QtWidgets.QSpinBox(self.gridLayoutWidget_3)
        self.spinBox.setProperty("value", 2)
        self.spinBox.setObjectName("spinBox")
        self.gridLayout_3.addWidget(self.spinBox, 0, 1, 1, 1)
        self.spinBox_3 = QtWidgets.QSpinBox(self.gridLayoutWidget_3)
        self.spinBox_3.setProperty("value", 2)
        self.spinBox_3.setObjectName("spinBox_3")
        self.gridLayout_3.addWidget(self.spinBox_3, 2, 1, 1, 1)
        self.spinBox_2 = QtWidgets.QSpinBox(self.gridLayoutWidget_3)
        self.spinBox_2.setProperty("value", 2)
        self.spinBox_2.setObjectName("spinBox_2")
        self.gridLayout_3.addWidget(self.spinBox_2, 1, 1, 1, 1)
        self.label = QtWidgets.QLabel(self.gridLayoutWidget_3)
        self.label.setObjectName("label")
        self.gridLayout_3.addWidget(self.label, 1, 0, 1, 1)
        self.label_2 = QtWidgets.QLabel(self.gridLayoutWidget_3)
        self.label_2.setObjectName("label_2")
        self.gridLayout_3.addWidget(self.label_2, 0, 0, 1, 1)
        self.label_3 = QtWidgets.QLabel(self.gridLayoutWidget_3)
        self.label_3.setObjectName("label_3")
        self.gridLayout_3.addWidget(self.label_3, 2, 0, 1, 1)
        self.formLayoutWidget = QtWidgets.QWidget(self.centralwidget)
        self.formLayoutWidget.setGeometry(QtCore.QRect(10, 50, 111, 61))
        self.formLayoutWidget.setObjectName("formLayoutWidget")
        self.formLayout = QtWidgets.QFormLayout(self.formLayoutWidget)
        self.formLayout.setContentsMargins(0, 0, 0, 0)
        self.formLayout.setObjectName("formLayout")
        self.label_4 = QtWidgets.QLabel(self.formLayoutWidget)
        self.label_4.setObjectName("label_4")
        self.formLayout.setWidget(0, QtWidgets.QFormLayout.LabelRole, self.label_4)
        self.lineEdit_17 = QtWidgets.QLineEdit(self.formLayoutWidget)
        self.lineEdit_17.setObjectName("lineEdit_17")
        self.formLayout.setWidget(0, QtWidgets.QFormLayout.FieldRole, self.lineEdit_17)
        self.label_5 = QtWidgets.QLabel(self.formLayoutWidget)
        self.label_5.setObjectName("label_5")
        self.formLayout.setWidget(1, QtWidgets.QFormLayout.LabelRole, self.label_5)
        self.lineEdit_18 = QtWidgets.QLineEdit(self.formLayoutWidget)
        self.lineEdit_18.setObjectName("lineEdit_18")
        self.formLayout.setWidget(1, QtWidgets.QFormLayout.FieldRole, self.lineEdit_18)
        self.textEdit = QtWidgets.QTextEdit(self.centralwidget)
        self.textEdit.setGeometry(QtCore.QRect(310, 10, 351, 561))
        self.textEdit.setObjectName("textEdit")
        self.formLayoutWidget_2 = QtWidgets.QWidget(self.centralwidget)
        self.formLayoutWidget_2.setGeometry(QtCore.QRect(10, 160, 281, 201))
        self.formLayoutWidget_2.setObjectName("formLayoutWidget_2")
        self.formLayout_2 = QtWidgets.QFormLayout(self.formLayoutWidget_2)
        self.formLayout_2.setContentsMargins(0, 0, 0, 0)
        self.formLayout_2.setObjectName("formLayout_2")
        self.label_7 = QtWidgets.QLabel(self.formLayoutWidget_2)
        self.label_7.setScaledContents(False)
        self.label_7.setObjectName("label_7")
        self.formLayout_2.setWidget(0, QtWidgets.QFormLayout.LabelRole, self.label_7)
        self.gridLayout_4 = QtWidgets.QGridLayout()
        self.gridLayout_4.setObjectName("gridLayout_4")
        self.checkBox_9 = QtWidgets.QCheckBox(self.formLayoutWidget_2)
        self.checkBox_9.setChecked(True)
        self.checkBox_9.setObjectName("checkBox_9")
        self.gridLayout_4.addWidget(self.checkBox_9, 0, 1, 1, 1)
        self.lineEdit_20 = QtWidgets.QLineEdit(self.formLayoutWidget_2)
        self.lineEdit_20.setObjectName("lineEdit_20")
        self.gridLayout_4.addWidget(self.lineEdit_20, 2, 2, 1, 1)
        self.lineEdit_25 = QtWidgets.QLineEdit(self.formLayoutWidget_2)
        self.lineEdit_25.setObjectName("lineEdit_25")
        self.gridLayout_4.addWidget(self.lineEdit_25, 2, 3, 1, 1)
        self.lineEdit_28 = QtWidgets.QLineEdit(self.formLayoutWidget_2)
        self.lineEdit_28.setObjectName("lineEdit_28")
        self.gridLayout_4.addWidget(self.lineEdit_28, 0, 2, 1, 1)
        self.checkBox_10 = QtWidgets.QCheckBox(self.formLayoutWidget_2)
        self.checkBox_10.setChecked(True)
        self.checkBox_10.setObjectName("checkBox_10")
        self.gridLayout_4.addWidget(self.checkBox_10, 2, 1, 1, 1)
        self.lineEdit_26 = QtWidgets.QLineEdit(self.formLayoutWidget_2)
        self.lineEdit_26.setObjectName("lineEdit_26")
        self.gridLayout_4.addWidget(self.lineEdit_26, 0, 3, 1, 1)
        self.formLayout_2.setLayout(0, QtWidgets.QFormLayout.FieldRole, self.gridLayout_4)
        self.label_8 = QtWidgets.QLabel(self.formLayoutWidget_2)
        self.label_8.setObjectName("label_8")
        self.formLayout_2.setWidget(1, QtWidgets.QFormLayout.LabelRole, self.label_8)
        self.gridLayout_5 = QtWidgets.QGridLayout()
        self.gridLayout_5.setObjectName("gridLayout_5")
        self.checkBox_11 = QtWidgets.QCheckBox(self.formLayoutWidget_2)
        self.checkBox_11.setChecked(True)
        self.checkBox_11.setObjectName("checkBox_11")
        self.gridLayout_5.addWidget(self.checkBox_11, 0, 1, 1, 1)
        self.lineEdit_21 = QtWidgets.QLineEdit(self.formLayoutWidget_2)
        self.lineEdit_21.setObjectName("lineEdit_21")
        self.gridLayout_5.addWidget(self.lineEdit_21, 2, 2, 1, 1)
        self.lineEdit_27 = QtWidgets.QLineEdit(self.formLayoutWidget_2)
        self.lineEdit_27.setObjectName("lineEdit_27")
        self.gridLayout_5.addWidget(self.lineEdit_27, 2, 3, 1, 1)
        self.lineEdit_29 = QtWidgets.QLineEdit(self.formLayoutWidget_2)
        self.lineEdit_29.setObjectName("lineEdit_29")
        self.gridLayout_5.addWidget(self.lineEdit_29, 0, 2, 1, 1)
        self.lineEdit_30 = QtWidgets.QLineEdit(self.formLayoutWidget_2)
        self.lineEdit_30.setObjectName("lineEdit_30")
        self.gridLayout_5.addWidget(self.lineEdit_30, 0, 3, 1, 1)
        self.checkBox_12 = QtWidgets.QCheckBox(self.formLayoutWidget_2)
        self.checkBox_12.setChecked(True)
        self.checkBox_12.setObjectName("checkBox_12")
        self.gridLayout_5.addWidget(self.checkBox_12, 2, 1, 1, 1)
        self.formLayout_2.setLayout(1, QtWidgets.QFormLayout.FieldRole, self.gridLayout_5)
        self.label_9 = QtWidgets.QLabel(self.formLayoutWidget_2)
        self.label_9.setObjectName("label_9")
        self.formLayout_2.setWidget(2, QtWidgets.QFormLayout.LabelRole, self.label_9)
        self.gridLayout_6 = QtWidgets.QGridLayout()
        self.gridLayout_6.setObjectName("gridLayout_6")
        self.checkBox_13 = QtWidgets.QCheckBox(self.formLayoutWidget_2)
        self.checkBox_13.setChecked(True)
        self.checkBox_13.setObjectName("checkBox_13")
        self.gridLayout_6.addWidget(self.checkBox_13, 0, 1, 1, 1)
        self.lineEdit_22 = QtWidgets.QLineEdit(self.formLayoutWidget_2)
        self.lineEdit_22.setObjectName("lineEdit_22")
        self.gridLayout_6.addWidget(self.lineEdit_22, 2, 2, 1, 1)
        self.lineEdit_31 = QtWidgets.QLineEdit(self.formLayoutWidget_2)
        self.lineEdit_31.setObjectName("lineEdit_31")
        self.gridLayout_6.addWidget(self.lineEdit_31, 2, 3, 1, 1)
        self.lineEdit_32 = QtWidgets.QLineEdit(self.formLayoutWidget_2)
        self.lineEdit_32.setObjectName("lineEdit_32")
        self.gridLayout_6.addWidget(self.lineEdit_32, 0, 2, 1, 1)
        self.lineEdit_33 = QtWidgets.QLineEdit(self.formLayoutWidget_2)
        self.lineEdit_33.setObjectName("lineEdit_33")
        self.gridLayout_6.addWidget(self.lineEdit_33, 0, 3, 1, 1)
        self.checkBox_14 = QtWidgets.QCheckBox(self.formLayoutWidget_2)
        self.checkBox_14.setChecked(True)
        self.checkBox_14.setObjectName("checkBox_14")
        self.gridLayout_6.addWidget(self.checkBox_14, 2, 1, 1, 1)
        self.formLayout_2.setLayout(2, QtWidgets.QFormLayout.FieldRole, self.gridLayout_6)
        self.verticalLayoutWidget = QtWidgets.QWidget(self.centralwidget)
        self.verticalLayoutWidget.setGeometry(QtCore.QRect(10, 370, 281, 91))
        self.verticalLayoutWidget.setObjectName("verticalLayoutWidget")
        self.verticalLayout = QtWidgets.QVBoxLayout(self.verticalLayoutWidget)
        self.verticalLayout.setContentsMargins(0, 0, 0, 0)
        self.verticalLayout.setObjectName("verticalLayout")
        self.pushButton_2 = QtWidgets.QPushButton(self.verticalLayoutWidget)
        self.pushButton_2.setObjectName("pushButton_2")
        self.verticalLayout.addWidget(self.pushButton_2)
        self.pushButton_3 = QtWidgets.QPushButton(self.verticalLayoutWidget)
        self.pushButton_3.setObjectName("pushButton_3")
        self.verticalLayout.addWidget(self.pushButton_3)
        self.checkBox = QtWidgets.QCheckBox(self.centralwidget)
        self.checkBox.setGeometry(QtCore.QRect(40, 120, 201, 41))
        self.checkBox.setObjectName("checkBox")
        self.label_10 = QtWidgets.QLabel(self.centralwidget)
        self.label_10.setGeometry(QtCore.QRect(10, 20, 121, 21))
        self.label_10.setObjectName("label_10")
        self.label_11 = QtWidgets.QLabel(self.centralwidget)
        self.label_11.setGeometry(QtCore.QRect(120, 10, 131, 41))
        self.label_11.setObjectName("label_11")
        MainWindow.setCentralWidget(self.centralwidget)
        self.menubar = QtWidgets.QMenuBar(MainWindow)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 671, 22))
        self.menubar.setObjectName("menubar")
        MainWindow.setMenuBar(self.menubar)
        self.statusbar = QtWidgets.QStatusBar(MainWindow)
        self.statusbar.setObjectName("statusbar")
        MainWindow.setStatusBar(self.statusbar)

        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "MainWindow"))
        self.lineEdit_13.setText(_translate("MainWindow", "-70"))
        self.lineEdit_16.setText(_translate("MainWindow", "20"))
        self.lineEdit_14.setText(_translate("MainWindow", "-70"))
        self.checkBox_7.setText(_translate("MainWindow", "y1"))
        self.lineEdit_15.setText(_translate("MainWindow", "20"))
        self.checkBox_8.setText(_translate("MainWindow", "y2"))
        self.label_6.setText(_translate("MainWindow", "Y"))
        self.label.setText(_translate("MainWindow", "poly2"))
        self.label_2.setText(_translate("MainWindow", "poly1"))
        self.label_3.setText(_translate("MainWindow", "poly3"))
        self.label_4.setText(_translate("MainWindow", "X"))
        self.lineEdit_17.setText(_translate("MainWindow", "SunX"))
        self.label_5.setText(_translate("MainWindow", "Y"))
        self.lineEdit_18.setText(_translate("MainWindow", "SunY"))
        self.label_7.setText(_translate("MainWindow", "X1"))
        self.checkBox_9.setText(_translate("MainWindow", "x11"))
        self.lineEdit_20.setText(_translate("MainWindow", "5"))
        self.lineEdit_25.setText(_translate("MainWindow", "16"))
        self.lineEdit_28.setText(_translate("MainWindow", "0.1"))
        self.checkBox_10.setText(_translate("MainWindow", "x12"))
        self.lineEdit_26.setText(_translate("MainWindow", "0.4"))
        self.label_8.setText(_translate("MainWindow", "X2"))
        self.checkBox_11.setText(_translate("MainWindow", "x21"))
        self.lineEdit_21.setText(_translate("MainWindow", "0"))
        self.lineEdit_27.setText(_translate("MainWindow", "5"))
        self.lineEdit_29.setText(_translate("MainWindow", "-14"))
        self.lineEdit_30.setText(_translate("MainWindow", "5"))
        self.checkBox_12.setText(_translate("MainWindow", "x22"))
        self.label_9.setText(_translate("MainWindow", "X3"))
        self.checkBox_13.setText(_translate("MainWindow", "x31"))
        self.lineEdit_22.setText(_translate("MainWindow", "-70"))
        self.lineEdit_31.setText(_translate("MainWindow", "20"))
        self.lineEdit_32.setText(_translate("MainWindow", "-70"))
        self.lineEdit_33.setText(_translate("MainWindow", "20"))
        self.checkBox_14.setText(_translate("MainWindow", "x32"))
        self.pushButton_2.setText(_translate("MainWindow", "Скоригувати X"))
        self.pushButton_3.setText(_translate("MainWindow", "Скоригувати Y"))
        self.checkBox.setText(_translate("MainWindow", "<- Дозвіл на коригування"))
        self.label_10.setText(_translate("MainWindow", "Назва Файлів"))
        self.label_11.setText(_translate("MainWindow", "Степені поліномів"))

In [4]:
class ExampleApp(QtWidgets.QMainWindow, Ui_MainWindow):
    def __init__(self):

        super().__init__()
        
        self.setupUi(self)  
        self.pushButton_2.clicked.connect(self.corX)
        self.pushButton_3.clicked.connect(self.corY)
        self.action = 0
        
    def corX(self):
        
        self.action = 'X'    
        response = self.request()
    
        self.textEdit.clear()
        #self.textEdit.append(str(get_results(response)))
        result = get_results(response)
        if result!= 'Empty':
        
            self.textEdit.append(str('x11 bound = '+ str(result[0])))
            self.textEdit.append(str('x12 bound = '+ str(result[1])))
            self.textEdit.append(str('x21 bound = '+ str(result[2])))
            self.textEdit.append(str('x22 bound = '+ str(result[3])))
            self.textEdit.append(str('x31 bound = '+ str(result[4])))
            self.textEdit.append(str('x32 bound = '+ str(result[5])))
        else:
            self.textEdit.append(str('Множини Парето не існує'))
            self.textEdit.append(str('Необхідно коригувати вхідні обмеження'))

    def corY(self):
        
        self.action = 'Y'
        response = self.request()
        
        self.textEdit.clear()
        result = get_results(response)
        
        self.textEdit.append(str('y1 bound = '+ str(result[0])))
        self.textEdit.append(str('y2 bound = '+ str(result[0])))
        
    def request(self):
        return {
    'X_values':pd.read_csv(str(self.lineEdit_17.text())+ '.txt'),
    'Y_values':pd.read_csv(str(self.lineEdit_18.text())+ '.txt'),
    'X':
    [
        {
            'min': float(self.lineEdit_28.text()),
            'max': float(self.lineEdit_26.text()),
            'correct': bool(self.checkBox_9.isChecked())        
        },
        {
            'min': float(self.lineEdit_20.text()),
            'max': float(self.lineEdit_25.text()),
            'correct': bool(self.checkBox_10.isChecked())
        },
        {
            'min': float(self.lineEdit_29.text()),
            'max': float(self.lineEdit_30.text()),
            'correct': bool(self.checkBox_11.isChecked())
        },
        {
            'min': float(self.lineEdit_21.text()),
            'max': float(self.lineEdit_27.text()),
            'correct': bool(self.checkBox_12.isChecked())
        },
        {
            'min': float(self.lineEdit_32.text()),
            'max': float(self.lineEdit_33.text()),
            'correct': bool(self.checkBox_13.isChecked())
        },
        {
            'min': float(self.lineEdit_22.text()),
            'max': float(self.lineEdit_31.text()),
            'correct': bool(self.checkBox_14.isChecked())
        },
    ],
    'Y': 
    [
        {
            'min': float(self.lineEdit_14.text()),
            'max': float(self.lineEdit_16.text()),
            'correct': bool(self.checkBox_7.isChecked())        
        },
        {
            'min': float(self.lineEdit_13.text()),
            'max': float(self.lineEdit_15.text()),
            'correct': bool(self.checkBox_8.isChecked())        
        },
    ],
    'poly': [self.spinBox.value(), self.spinBox_2.value(), self.spinBox_3.value()],
    'action':str(self.action)
}

In [5]:
def main():
    app = QtWidgets.QApplication(sys.argv)  
    window = ExampleApp()  
    window.show()  
    app.exec_()  

if __name__ == '__main__':  
    main() 